# Bootstrap optimism-corrected validation

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats
import scipy.special
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Original sample

In [2]:
n = 500
df = pd.DataFrame()
df['x'] = np.random.randn(n)
df['z'] = -1.0 + 1.0 * df.x
df['p'] = scipy.special.expit(df.z)
df['y'] = scipy.stats.bernoulli.rvs(df.p)

df.describe()

,x,z,p,y
count,500.000000,500.000000,500.000000,500.000000
mean,-0.021771,-1.021771,0.302191,0.300000
std,1.044361,1.044361,0.185487,0.458717
min,-3.420272,-4.420272,0.011888,0.000000
25%,-0.788795,-1.788795,0.143221,0.000000
50%,0.016913,-0.983087,0.272280,0.000000
75%,0.675638,-0.324362,0.419613,1.000000
max,3.271772,2.271772,0.906512,1.000000


## Bootstrap validate logit model

In [3]:
# McFadden's Pseudo R^2
def pr2(obs, null_probs, model_probs):
    ll_null = np.sum(scipy.stats.bernoulli.logpmf(obs, null_probs))
    ll_model = np.sum(scipy.stats.bernoulli.logpmf(obs, model_probs))
    return 1 - ll_model / ll_null

### Estimate optimism

In [4]:
B = 2000
optimism = np.zeros(B)

for bi in range(B):
    b = df.sample(frac=1.0, replace=True)
    logit = smf.glm('y ~ x', family=sm.families.Binomial(), data=b).fit()
    
    null_probs = np.repeat(b.y.mean(), len(df))
    pr2_boot = pr2(b.y, null_probs, logit.predict(b))
    pr2_orig = pr2(df.y, null_probs, logit.predict(df))
    optimism[bi] = pr2_boot - pr2_orig

### Optimism-corrected Pseudo R^2

In [5]:
final = smf.glm('y ~ x', family=sm.families.Binomial(), data=df).fit()
pr2_apparent = pr2(df.y, np.repeat(df.y.mean(), len(df)), final.predict(df))

print('Mean optimism', np.mean(optimism))
print('Apparent PR2', pr2_apparent)
print('Optimism-corrected PR2', pr2_apparent - np.mean(optimism))

Mean optimism 0.003799143286195928
Apparent PR2 0.15379689114238015
Optimism-corrected PR2 0.14999774785618422
